# Aries Basic Controller Example - Bob
## DID Exchange - Invitee

In this notebook we'll be responding to an invitation from Alice through our Agents. This notebook has the following phases:

<b>Begin with the Alice notebook ([localhost:8888]("http://localhost:8888"))</b>


6. Pull in dependencies
7. Instatiate the controller for our Agent
8. Set up a message listener running as a service on our controller
9. Paste the invitation from the Alice notebook into the invitation variable here
10. Accept the invitation

<b>Carry on at the Alice side</b>
    
14. Send Trust Ping to the Alice


### 6. Pull in dependencies


In [1]:
%autoawait
import time
import asyncio

IPython autoawait is `on`, and set to use `asyncio`


### 7. Instatiate the controller for our Agent

In [2]:
from aries_basic_controller.aries_controller import AriesAgentController
    
WEBHOOK_HOST = "0.0.0.0"
WEBHOOK_PORT = 8052
WEBHOOK_BASE = ""
ADMIN_URL = "http://bob-agent:8051"

agent_controller = AriesAgentController(webhook_host=WEBHOOK_HOST, webhook_port=WEBHOOK_PORT,
                                       webhook_base=WEBHOOK_BASE, admin_url=ADMIN_URL, connections=True)

### 8. Set up a listener for basicmessages events emitted by the controller as it receives webhooks

In [3]:
def messages_handler(payload):
    connection_id = payload["connection_id"]
    print("Handle messages ", payload, connection_id)

message_listener = {
    "handler": messages_handler,
    "topic": "basicmessages"
}

loop = asyncio.get_event_loop()
loop.create_task(agent_controller.listen_webhooks())

agent_controller.register_listeners([message_listener], defaults=True)

### 9. Paste the invitation from the Alice notebook into the invitation variable here;

In [4]:
#Paste in invitation from Alice agent
invite = {'connection_id': 'f6bfb182-8c02-4303-9f0d-560d457b2273', 'invitation': {'@type': 'did:sov:BzCbsNYhMrjHiqZDTUASHg;spec/connections/1.0/invitation', '@id': '2342f444-90d3-4163-8422-2e09fd68d7bf', 'serviceEndpoint': 'http://172.17.0.1:8020', 'label': 'Alice', 'recipientKeys': ['E8Mtk7Daj9uPyNDxUfDkyHPYVaRA1AXuNH8d8BmQNsiv']}, 'invitation_url': 'http://172.17.0.1:8020?c_i=eyJAdHlwZSI6ICJkaWQ6c292OkJ6Q2JzTlloTXJqSGlxWkRUVUFTSGc7c3BlYy9jb25uZWN0aW9ucy8xLjAvaW52aXRhdGlvbiIsICJAaWQiOiAiMjM0MmY0NDQtOTBkMy00MTYzLTg0MjItMmUwOWZkNjhkN2JmIiwgInNlcnZpY2VFbmRwb2ludCI6ICJodHRwOi8vMTcyLjE3LjAuMTo4MDIwIiwgImxhYmVsIjogIkFsaWNlIiwgInJlY2lwaWVudEtleXMiOiBbIkU4TXRrN0Rhajl1UHlORHhVZkRreUhQWVZhUkExQVh1Tkg4ZDhCbVFOc2l2Il19'}

### 10. Accept the invitation, then move to Alice's notebook

In [5]:
# Receive Invitation
response = await agent_controller.connections.accept_connection(invite["invitation"])
# Print out accepted Invite and Alice's connection ID
print("Connection", response)
alice_id = response["connection_id"]


Connection {'state': 'request', 'invitation_mode': 'once', 'accept': 'manual', 'their_label': 'Alice', 'connection_id': '90383918-3009-4ced-b420-163c0a0ceead', 'my_did': 'PP3pUET3rqfyx3n6NuHSpR', 'invitation_key': 'E8Mtk7Daj9uPyNDxUfDkyHPYVaRA1AXuNH8d8BmQNsiv', 'initiator': 'external', 'request_id': '2635322f-5455-4426-994e-73037eeb043d', 'routing_state': 'none', 'updated_at': '2020-07-07 14:57:49.579538Z', 'created_at': '2020-07-07 14:57:49.563813Z'}


### 14. Send Trust Ping to the Alice

In [6]:
# Do a trust ping to ensure connection is accepted by alice
trust_ping = await agent_controller.messaging.trust_ping(alice_id,"hello")
print("TUST PING TO ACTIVATE CONNECTION - Bob -> Alice")
print(trust_ping)   

TUST PING TO ACTIVATE CONNECTION - Bob -> Alice
{'thread_id': '757e2cc1-644f-4964-b632-c3fb2bbd5689'}


#### 14.1 Check if connection state is active

In [6]:
# Print connection list
connection = await agent_controller.connections.get_connection(alice_id)
print("Alice AGENT CONNECTION")
print("State", connection["state"])

Alice AGENT CONNECTION
State active


#### 14.2 Send a basic message over e2e DIDComm channel

In [10]:
#send a basic message
message = await agent_controller.messaging.send_message(alice_id,"hello from bob world!")
print("BASIC MESSAGE - Bob -> Alice")
print(message)

BASIC MESSAGE - Bob -> Alice
{}


In [6]:
response = await agent_controller.issuer.get_records()
print(response)

{'results': [{'thread_id': 'f176df38-f9a6-46f7-a894-44ebfc633519', 'trace': False, 'credential_definition_id': 'PQRXDxdGqQGSZ8z69p4xZP:3:CL:10:default', 'auto_issue': False, 'auto_remove': True, 'credential_proposal_dict': {'@type': 'did:sov:BzCbsNYhMrjHiqZDTUASHg;spec/issue-credential/1.0/propose-credential', '@id': 'bbeafd1a-4e09-4981-9200-9d522c7031f1', 'comment': 'create automated credential exchange', 'schema_id': 'PQRXDxdGqQGSZ8z69p4xZP:2:open_mined_contributor:0.0.1', 'cred_def_id': 'PQRXDxdGqQGSZ8z69p4xZP:3:CL:10:default', 'credential_proposal': {'@type': 'did:sov:BzCbsNYhMrjHiqZDTUASHg;spec/issue-credential/1.0/credential-preview', 'attributes': [{'name': 'name', 'value': 'Bob'}, {'name': 'skill', 'value': 'researcher'}]}}, 'state': 'offer_received', 'credential_offer': {'schema_id': 'PQRXDxdGqQGSZ8z69p4xZP:2:open_mined_contributor:0.0.1', 'cred_def_id': 'PQRXDxdGqQGSZ8z69p4xZP:3:CL:10:default', 'key_correctness_proof': {'c': '97136431841517236056383965410503472951566378926335